In [1]:
import jieba
import numpy as np
import yaml
from gensim.corpora.dictionary import Dictionary
from gensim.models.word2vec import Word2Vec
import torch

D:\Conda\envs\d2l\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
maxlen = 20

In [ ]:
def create_dictionaries(model=None, combined=None):
    """创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引"""
    if (combined is not None) and (model is not None):
        if os.path.exists(Config.combined_path) and os.path.exists(Config.w2vec_path) and os.path.exists(
                Config.w2indx_path):  # 已写入
            print("加载词典")
            w2indx_json = codecs.open(Config.w2indx_path, 'r', encoding=Config.encoding)
            w2vec_json = codecs.open(Config.w2vec_path, 'r', encoding=Config.encoding)
            combined_json = codecs.open(Config.combined_path, 'r', encoding=Config.encoding)

            w2indx = json.load(w2indx_json)
            W2VEC = json.load(w2vec_json)  # 加载的数组为list，需转换为numppy数组
            combined = np.loadtxt(combined_json, dtype=int)

            # 转换
            w2vec = dict()
            for key, value in W2VEC.items():
                w2vec[key] = np.asarray(value)

            w2indx_json.close()
            w2vec_json.close()
            combined_json.close()

            return w2indx, w2vec, combined
        else:
            gensim_dict = Dictionary()  # 创建一个空的词典,构建 word<->id 映射
            gensim_dict.doc2bow(list(model.wv.index_to_key),
                                allow_update=True)  # 构建词袋，每个单词对应一个id，词袋中的单词不重复
            w2indx = {v: k + 1 for k, v in gensim_dict.items()}  # 所有频数超过10的词语的索引字典
            w2vec = {word: model.wv[word] for word in w2indx.keys()}  # 所有频数超过10的词语的词向量字典
            W2VEC = {word: model.wv[word].tolist() for word in w2indx.keys()}  # 将numpy数组转换为list存储

            def parse_dataset(combined):
                """将combined中的数据转换为索引表示"""
                data = []
                for sentence in combined:
                    new_txt = []
                    for word in sentence:
                        try:
                            new_txt.append(w2indx[word])
                        except:
                            new_txt.append(0)
                    new_txt=torch.Tensor(new_txt[:maxlen]).int()
                    data.append(new_txt)
                return data

            combined = parse_dataset(combined)  # 将combined中的数据转换为索引表示
            combined = nn.utils.rnn.pad_sequence(combined, batch_first=True, padding_value=0)
            # 每个句子所含词语对应的索引，所以句子中含有频数小于10的词语，索引为0

            w2indx_json = codecs.open(Config.w2indx_path, 'w', encoding=Config.encoding)
            w2vec_json = codecs.open(Config.w2vec_path, 'w', encoding=Config.encoding)
            combined_json = codecs.open(Config.combined_path, 'w', encoding=Config.encoding)

            json.dump(w2indx, w2indx_json)
            json.dump(W2VEC, w2vec_json)
            np.savetxt(combined_json, combined)  # numpy.ndarrayi

            w2indx_json.close()
            w2vec_json.close()
            combined_json.close()

            return w2indx, w2vec, combined
    else:
        print('No data provided...')

In [ ]:
def input_transform(string):
    words = jieba.lcut(string)
    words = np.array(words).reshape(1, -1)
    model = Word2Vec.load('word2vec/Word2vec_model.pkl')
    _, _, combined = create_dictionaries(model, words)
    return combined